In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
tf.random.set_seed(1234)
from sklearn.model_selection import train_test_split 

2023-09-29 15:50:39.836391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd

In [3]:
import glob

folder_path = "/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling"  # Replace with the actual path to your folder

# Use glob to get a list of all files in the folder
files = glob.glob(folder_path + "/*")

# Now, the 'files' variable contains a list of file paths in the specified folder
print(files)


['/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling1.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling2.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling3.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling7.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling6.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling4.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling5.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling13.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling12.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling10.csv', '/Users/anishayyagari/Documents/tester/pointnet implementation/data/falling/falling11.

In [4]:
def parse_dataset(num_points=400):

    train_points = []
    train_labels = []
    class_map = {}
    folder_path = "/Users/anishayyagari/Documents/tester/pointnet implementation/data"
    folders = glob.glob(folder_path + "/*")

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = folder.split("/")[-1]
        # gather all files

        train_files = glob.glob(os.path.join(folder_path,folder)+"/*")

        for f in train_files:


            # Use the random indices to select 200 random rows from the original array
            train_points.append(pd.read_csv(f).set_index('Unnamed: 0').sample(num_points).values)
            train_labels.append(i)

    return (
        np.array(train_points),
        np.array(train_labels),
        class_map,
    )

In [5]:
train_points = []
train_labels = []
class_map = {}
folder_path = "/Users/anishayyagari/Documents/tester/pointnet implementation/data"
folders = glob.glob(folder_path + "/*")

for i, folder in enumerate(folders):
    print("processing class: {}".format(os.path.basename(folder)))
  # store folder name with ID so we can retrieve later
    class_map[i] = folder.split("/")[-1]
  # gather all files

    train_files = glob.glob(os.path.join(folder_path,folder)+"/*")

    for f in train_files:
        train_points.append(pd.read_csv(f).set_index('Unnamed: 0'))
        train_labels.append(i)

processing class: lying_down
processing class: falling
processing class: standing
processing class: sitting


In [6]:
train_points[0].shape

(275, 3)

In [7]:
NUM_POINTS = 150
NUM_CLASSES = 4
BATCH_SIZE = 32
train_points,train_labels,class_map = parse_dataset(NUM_POINTS)
test_size = 0.27
X_train, X_test, Y_train, y_test = train_test_split(train_points, train_labels, test_size=test_size, random_state=123)

processing class: lying_down
processing class: falling
processing class: standing
processing class: sitting


In [8]:
train_points.shape

(364, 150, 3)

In [9]:
import tf_util

In [10]:
def get_model(point_cloud, is_training, bn_decay=None):
    """ Classification PointNet, input is BxNx3, output Bx40 """
    batch_size = point_cloud.get_shape()[0]
    num_point = point_cloud.get_shape()[1]
    end_points = {}
    input_image = tf.expand_dims(point_cloud, -1)

    # Point functions (MLP implemented as conv2d)
    net = tf_util.conv2d(input_image, 64, [1,3],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv1', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv2', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv3', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv4', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 1024, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv5', bn_decay=bn_decay)

    # Symmetric function: max pooling
    net = tf_util.max_pool2d(net, [num_point,1],
                             padding='VALID', scope='maxpool')

    # MLP on global point cloud vector
    net = tf.reshape(net, [batch_size, -1])
    net = tf_util.fully_connected(net, 512, bn=True, is_training=is_training,
                                  scope='fc1', bn_decay=bn_decay)
    net = tf_util.fully_connected(net, 256, bn=True, is_training=is_training,
                                  scope='fc2', bn_decay=bn_decay)
    net = tf_util.dropout(net, keep_prob=0.7, is_training=is_training,
                          scope='dp1')
    net = tf_util.fully_connected(net, 4, activation_fn=None, scope='fc3')

    return net, end_points


def get_loss(pred, label,end_points):
    """ pred: B*NUM_CLASSES,
        label: B, """
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=label)
    classify_loss = tf.reduce_mean(loss)
    tf.summary.scalar('classify loss', classify_loss)
    return classify_loss

In [24]:
BATCH_SIZE = 1

In [25]:
tf.compat.v1.disable_eager_execution()
point_cloud = tf.compat.v1.placeholder(tf.float32, shape=(BATCH_SIZE, NUM_POINTS, 3))
label = tf.compat.v1.placeholder(tf.int32, shape=(BATCH_SIZE,))
is_training = tf.constant(True)  # Set to True for training, False for inference


In [26]:
pred, end_points = get_model(point_cloud, is_training)

In [27]:
learning_rate = 0.001  # Adjust this as needed
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(get_loss(pred, label, end_points))

In [28]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [29]:
train_points.shape

(364, 150, 3)

In [30]:
saver = tf.compat.v1.train.Saver()
num_epochs = 50
for epoch in range(num_epochs):
    # Loop through the training dataset in mini-batches
    for i in range(0, len(X_train), BATCH_SIZE):
        batch_data = X_train[i:i + BATCH_SIZE]
        batch_labels = Y_train[i:i + BATCH_SIZE]

        # Feed batch data into the model and run the training operation
        feed_dict = {
            point_cloud: batch_data,
            label: batch_labels,
            is_training: True  # Set to True for training
        }
        _, loss_val = sess.run([train_op, get_loss(pred, label, end_points)], feed_dict=feed_dict)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss_val:.4f}")

    # Perform validation at the end of each epoch
    val_loss = 0.0
    num_val_batches = len(X_test) // BATCH_SIZE
    for j in range(0, len(X_test), BATCH_SIZE):
        val_batch_data = X_test[j:j + BATCH_SIZE]
        val_batch_labels = y_test[j:j + BATCH_SIZE]

        feed_dict = {
            point_cloud: val_batch_data,
            label: val_batch_labels,
            is_training: False  # Set to False for validation
        }
        val_loss += sess.run(get_loss(pred, label, end_points), feed_dict=feed_dict)

    val_loss /= num_val_batches
    print(f"Validation Loss: {val_loss:.4f}")

    # Save the model's weights after each epoch (or at any desired frequency)
    if (epoch + 1) % 1 == 0:
        save_path = saver.save(sess, "model_weights/model.ckpt")
        print(f"Saved model weights to {save_path}")

Epoch 1/50, Loss: 1.4364
Validation Loss: 2398.8808


NameError: name 'saver' is not defined